**Knee Detection Model for Lateral Knee Joint (Main Jupyter Notebook)**


Version 1.1

By Lu Yik Ho

**Step 1: Program Requirements**

This program requires preinstalled modules. This step provides a guide on the required modules.


**Step 1.3**      

Confirm Module Installation 

In [ ]:
# Import all required modules.

# Run this code box once to ensure all modules are installed.

import torch

gpu_avail = torch.cuda.is_available()
print(f"Is the GPU available? {gpu_avail}")
torch.cuda.get_device_name(0)


import pydicom
import os
import glob
import numpy as np
import json
import torch.utils.data
import torchvision
from torchvision import transforms
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import shutil
import ultralytics
import cv2
from ultralytics import YOLO
from sklearn.model_selection import train_test_split
import yaml


print("All modules imported!")

**Step 2: Preparing Images for LabelMe**

These steps mainly consist of processing the images retrieved from DICOM files. These steps aid in labelling bounding boxes with LabelMe, and in the training the YOLO model.

You should run this step for both preparing training and inference images.

**Step 2.1:**

In [ ]:
# The original file in dataset is DICOM, we need to convert to PNG so that we can check the each step output easily

from KneeDetectionLateral_ImageProcessing import dicom_to_png

# Input path to DICOM Files
dicom_files_lateralL = glob.glob("./DICOM Files/**/**/**/**/LL**/**")
dicom_files_lateralR = glob.glob("./DICOM Files/**/**/**/**/RL**/**")

count = 0 

# Convert all DICOM files to PNG
for file in dicom_files_lateralL:
    if not file.endswith(".png") and not file.endswith(".json") and not file.endswith(".txt"):
        dicom_to_png(file)
        count = count + 1

print("LL Images: {}".format(count))
count = 0 

# Convert all DICOM files to PNG
for file in dicom_files_lateralR:
    if not file.endswith(".png") and not file.endswith(".json") and not file.endswith(".txt"):
        dicom_to_png(file)
        count = count + 1

print("RL Images: {}".format(count))

**Step 2.2:**

In [ ]:
# Flip all right lateral images to left lateral images

from KneeDetectionLateral_ImageProcessing import flip_png

# Input target PNG files
png_files = glob.glob("./DICOM Files/**/**/**/**/RL**/*.png")

# Flip all PNG files
for file in png_files:
    
    # Prevents flipping twice
    if not file.endswith("_C.png") and not file.endswith("_F.png") and not file.endswith("_N.png"):
      flip_png(file)

**Step 2.3:**

In [ ]:
# This code should be used specifically for the lateral images to remove the second joint.

from KneeDetectionLateral_ImageProcessing import split_png_lateral

# Input target PNG files
png_files = glob.glob("./DICOM Files/**/**/**/**/**/*.png")

# Adjust percentage of pixels to be cropped here:
crop_value = 0.3

# Adjust which side of the image should be cropped
crop_side = "Left"

# Crop all PNG files
for file in png_files:
    if not file.endswith("_C.png") and not file.endswith("_N.png"):
        split_png_lateral(file, crop_value, crop_side)
        

**Step 2.4:**

In [ ]:
# Normalise images 

from KneeDetectionLateral_ImageProcessing import normal_img

# Input target PNG files
input_files_L = glob.glob("./DICOM Files/**/**/**/**/LL**/*_C.png")

input_files_R = glob.glob("./DICOM Files/**/**/**/**/RL**/*_F_C.png")

# Input output directory
output_dir = "./Images for Labeling"

for file in input_files_L:
    if file.endswith("_C.png"):
      normal_img(file, True, output_dir)

for file in input_files_R:
    if file.endswith("_F_C.png"):
      normal_img(file, True, output_dir)

**Step 3: Creating Bounding Boxes with LabelMe**

LabelMe is the main program used to create the bounding box of the region of interest. These steps provide a guide to the specific method to use LabelMe for this program.

This Jupyter Notebook is not required for these steps.

**Step 4: Creating YOLO Required Dataset**

These steps convert the current images and labels into a dataset required for training a YOLO model.


**Step 4.2:**

In [ ]:
# Converting LabelMe bounding boxes to YOLO bounding boxes
# LabelMe: (x-min, y-min, x-max, y-max) in .json file
# YOLO: (x1, y1, x2, y2...) in .txt file
# LabelMe and YOLO shares same coordinate system, but YOLO requires normalisation of coordinates

from KneeDetectionLateral_TrainYoloModel import LMtoYOLO

# Input target JSON files
json_files = glob.glob("./Training Images/**")

# Input output directory
output_dir = "./Training Labels"

# Run conversion for all files
for file in json_files:
    if file.endswith(".json"):
        LMtoYOLO(file, output_dir)


**Step 4.3:**

In [ ]:
# Splits the current data from directories into training and testing data, and saves data to YOLO required directory format

from KneeDetectionLateral_TrainYoloModel import YOLOTTSplit

# Input and Output Directories
image_dir = "./Training Images"
txt_dir = "./Training Labels"
target_dir = "./YOLO Dataset"

# Percentage of data to be used at testing
test_section = 0.20

YOLOTTSplit(image_dir, txt_dir, target_dir, test_section)

**Step 4.4**

In [ ]:
# Writing the required .yaml file required for YOLO

from KneeDetectionLateral_TrainYoloModel import createYaml

# Path to YOLO dataset
path = "./YOLO Dataset"

# Names of classes
names = ["Knee Joint"]

# Relative path to validation images
train = "images/train"

# Relative path to validation images
val = "images/val"

# Number of classses
nc = 1 

createYaml(path, train, val, nc, names, path)

**Step 5: Fine-tuning YOLO Model**

These steps fine-tune a YOLO11 nano segmentation model, along with a guide to understand the output metrics of the training.


**Step 5.1/5.2:**

In [ ]:
# Trains a YOLO model with the dataset. 

# Please make sure the .yaml file is directly under this directory!

from KneeDetectionLateral_TrainYoloModel import trainModel    

# The directory to your YOLO dataset
dataset_dir = "./YOLO Dataset"

# Path to your model. Put None if training brand new model
model_path = None

# Set number of epochs
epoch_num = 250

# Toggle simplified output
simple_output = True

# 1 simple output per number of epochs
simple_output_per_epoch = 10


trainModel(dataset_dir, model_path, epoch_num, simple_output, simple_output_per_epoch)



**Step 5.3:**

In [ ]:
from KneeDetectionLateral_TrainYoloModel import extraResultPlots

json_file = "./YOLO Dataset/output_data.json"

extraResultPlots(json_file)

**Step 6: Inference with Tuned Model**

These steps run an inference test with completely new images, testing the performance and capabilities of the trained model. 


**Step 6.1:**

In [ ]:
# Converting Inference .json files

from KneeDetectionLateral_TrainYoloModel import LMtoYOLO

# Input target JSON files
json_files = glob.glob("./Inference Images/**")

# Input output directory
output_dir = "./Inference Labels"

# Run conversion for all files
for file in json_files:
    if file.endswith(".json"):
        LMtoYOLO(file, output_dir)

**Step 6.2:**

In [ ]:
# Main inference program:

from KneeDetectionLateral_TrainYoloModel import inferenceCheck, calculatePredictBox, calculateTruthBox, plotBoundingBox, manualDice, manualIoU, manual_mAP
        
# Load trained model 
model = YOLO("./runs/segment/train/weights/best.pt")

# Input target PNG files
img_files = glob.glob("./Inference Images/**")

# Input labels directory
txt_dir = "./Inference Labels"

# Toggle displaying each prediction image in a new window
window_per_img = False

# Initialize matploblib figure (sizse: w,h in inches)
figure = plt.figure(figsize=(32, 48))

# Set the format of subplots. The number of rows should be a factor of the number of images
rows = 1

cols = int(len(results)/rows)
figure.subplots(rows,cols)

# Create lists for final processing
IoU_list = []
Dice_list = []

# Run results through model
results = inferenceCheck(model, img_files, False)

# Run once per image, with incrementing index
for index, result in enumerate(results):

    # Swtich to correct subplot
    subplot = plt.subplot(rows,cols,index+1)

    # Display image in subplot
    subplot.imshow(Image.open(result.path), cmap='gray', vmin=0, vmax=255)

    # Retrieve information from predicted result
    predict_box = calculatePredictBox(result)

    # Plot predicted bounding box
    plotBoundingBox(float(predict_box["x_min"]), float(predict_box["y_min"]), float(predict_box["box_width"]), float(predict_box["box_height"]), subplot, "r")

    # Retrieve information from ground truth
    truth_box = calculateTruthBox(result, txt_dir, predict_box["img_width"], predict_box["img_height"])
    print(truth_box)

    # Plot ground truth bounding box
    plotBoundingBox(truth_box["x_min"], truth_box["y_min"], truth_box["box_width"], truth_box["box_height"], subplot, "g")

    # Calculate intersection over union 
    IoU_score =  manualIoU(predict_box, truth_box)
    IoU_list.append(IoU_score)
    mAP50, mAP50_95 = manual_mAP(IoU_list)

    # Calculate dice coefficient
    Dice_score = manualDice(predict_box, truth_box)
    Dice_list.append(Dice_score)


    # Add labels to plot
    plt.title(truth_box["file_name"] + ".png")
    plt.xlabel("IoU Score: {}\nDice Score: {}".format(IoU_score, Dice_score))

# Display plotted figure
plt.show()
print("Inference check complete!")

# Output statistics
print("Number of Images: {}\n".format(len(results)))
print("Highest IoU (Intersection over Union): {}".format(max(IoU_list)))
print("Lowest IoU (Intersection over Union): {}".format(min(IoU_list)))
print("Average IoU (Intersection over Union): {}".format(sum(IoU_list)/len(IoU_list)))
print("mAP50: {}".format(mAP50))
print("mAP50-95: {}".format(mAP50_95))
print("Highest Dice Coefficient: {}".format(max(Dice_list)))
print("Lowest Dice Coefficient: {}".format(min(Dice_list)))
print("Average Dice Coefficient: {}".format(sum(Dice_list)/len(Dice_list)))



**Step 7: Cropping Images with Tuned Model**

If inference yielded successful or expected results, the model can be used to crop images to their region of interest. These steps crop the image and process the image for the final output.


**Step 7.1:**

In [ ]:
from KneeDetectionLateral_ImageProcessing import predict_img, crop_img

from KneeDetectionLateral_TrainYoloModel import calculatePredictBox, calculateSquareBox

# Input target PNG files
model = YOLO("./runs/segment/train/weights/best.pt")

# Input target PNG files
img_files = glob.glob("./Processing Images/**")

# Input output directory
out_dir = "./Processing Images Cropped"

# The part of the bounding box to be changed to create a square bounding box. Enter "None" to bypass.
square_from = "Bottom"

for file in img_files:
    result = predict_img(model, file)

    predict_box = calculatePredictBox(result)

    adjust_box = calculateSquareBox(predict_box, square_from)

    crop_img(file, out_dir, float(adjust_box["x_min"]), float(adjust_box["y_min"]), float(adjust_box["box_width"]), float(adjust_box["box_height"]))
    

**Step 7.2:**

In [ ]:
# Normalise output images

from KneeDetectionLateral_ImageProcessing import normal_img

img_files = glob.glob("./Processing Images Cropped/**")
output_dir = "./Processing Images Normalised"

for file in img_files:
    if file.endswith(".png"):
      normal_img(file, True, output_dir)

In [ ]:
# Apply CLAHE Normalisation on output images

from KneeDetectionLateral_ImageProcessing import apply_clahe

img_files = glob.glob("./Processing Images Normalised/**")
output_dir = "./Processing Images CLAHE"

for file in img_files:
    if file.endswith(".png"):
      apply_clahe(file, output_dir)

**Achknowledgement**

@software{yolov8_ultralytics,
  author = {Glenn Jocher and Ayush Chaurasia and Jing Qiu},
  title = {Ultralytics YOLOv8},
  version = {8.0.0},
  year = {2023},
  url = {https://github.com/ultralytics/ultralytics},
  orcid = {0000-0001-5950-6979, 0000-0002-7603-6750, 0000-0003-3783-7069},
  license = {AGPL-3.0}
}